In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
import re
import time
import math

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36'}

response = requests.get(url='https://wx.lianjia.com/zufang/',headers=headers)

url = 'https://wx.lianjia.com/zufang/{}/pg{}/'

In [175]:
area_dic = {
    '梁溪':'liangxi',
    '滨湖':'binhu',
    '锡山':'xishan',
    '惠山':'huishan',
    '新吴':'xinwu',
    '宜兴市':'yixingshi',
    '江阴市':'jiangyinshi'
}

In [4]:
def re_match(re_pattern,string,errif=None):
    try:
        return re.findall(re_pattern,string)[0].strip()
    except IndexError:
        return errif

In [176]:
data = pd.DataFrame()

for key,value in area_dic.items():
    start_url = 'https://wx.lianjia.com/zufang/{}/'.format(value)
    html = requests.get(url=start_url,headers=headers).content.decode('utf-8')
    pattern = re.compile('<span class="content__title--hl">(.*?)</span>.*无锡',re.S)
    house_num = re.findall(pattern,html)[0].strip()
    print('{}:租房共{}套'.format(key,house_num))
    time.sleep(1)
    # 页面限制 每个区最多100页3000条信息
    total_page = int(math.ceil(min(3000,int(house_num))/30.0))

    for i in tqdm(range(total_page),desc=key):
        html = requests.get(url=url.format(value,i+1),headers=headers).content.decode('utf-8')
        soup = BeautifulSoup(html,'lxml')

        info = soup.find_all(class_ = 'content__list--item--main')
        for item in info:
            info_dic = {}
            info_dic['area'] = key
            info_dic['title'] = re.findall(re.compile('.*<p class="content__list--item--title twoline">.*?href=.*?>(.*?)</a>',re.S),str(item))[0].strip()
            try:
                housesize = re.findall(re.compile('<p class="content__list--item--des">.*?<i>/</i>(.*?)㎡',re.S),str(item))[0].strip()
            except:
                housesize = re.findall(re.compile('<p class="content__list--item--des">(.*?)㎡',re.S),str(item))[0].strip()
            
            if '-' in housesize:
                low,high = housesize.split('-')
                info_dic['house_size'] = (int(low)+int(high))/2
            else:
                info_dic['house_size'] = int(housesize)

            try:
                housetypelist = re.findall(re.compile('<p class="content__list--item--des">.*?<i>/</i>.*?(\d)室(\d)厅(\d)卫.*?</p>',re.S),str(item))[0]   
            except:
                housetypelist = [0,0,0]
            info_dic['bedroon_num'] = int(housetypelist[0])
            info_dic['hall_num'] = int(housetypelist[1])
            info_dic['bathroom_num'] = int(housetypelist[2])
                
            price = re.findall(re.compile('<span class="content__list--item-price"><em>(.*?)</em>.*?</span>',re.S),str(item))[0]
            if '-' in price:
                low,high = price.split('-')
                info_dic['price'] = (int(low)+int(high))/2
            else:
                info_dic['price'] = float(price)
            #独栋·冠寓
            typeplace = (info_dic['title'].split(' ')[0]).split('·')
            info_dic['type'] = typeplace[0]
            info_dic['place'] = typeplace[1]

            if data.empty:
                data = pd.DataFrame(info_dic,index=[0])
            else:
                data = data.append(info_dic,ignore_index=True)

梁溪:租房共5499套
梁溪: 100%|██████████| 100/100 [01:24<00:00,  1.18it/s]
滨湖:租房共5102套
滨湖: 100%|██████████| 100/100 [01:32<00:00,  1.08it/s]
锡山:租房共1777套
锡山: 100%|██████████| 60/60 [00:58<00:00,  1.03it/s]
惠山:租房共1687套
惠山: 100%|██████████| 57/57 [01:02<00:00,  1.10s/it]
新吴:租房共2851套
新吴: 100%|██████████| 96/96 [02:43<00:00,  1.71s/it]
宜兴市:租房共4套
宜兴市: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it]
江阴市:租房共746套
江阴市: 100%|██████████| 25/25 [00:43<00:00,  1.73s/it]


In [177]:
data.head()

,area,title,house_size,bedroon_num,hall_num,bathroom_num,price,type,place
0,梁溪,整租·金星苑 1室1厅 南,65.0,1,1,1,1300.0,整租,金星苑
1,梁溪,独栋·柚米国际社区 柚米(无锡三阳广场店) 一居清新风 1室1厅,19.5,1,1,1,2072.5,独栋,柚米国际社区
2,梁溪,整租·华新苑 3室1厅 南,125.0,3,1,1,600.0,整租,华新苑
3,梁溪,独栋·柚米国际社区 酒吧街店 宜家风 1室1厅,27.0,1,1,1,2180.0,独栋,柚米国际社区
4,梁溪,整租·华新苑 1室2厅 南,48.0,1,2,1,1200.0,整租,华新苑


In [9]:
#如果价格是区间的,按平均价计算
# def get_aver(data):
#     if isinstance(data, str) and '-' in data:
#         low, high = data.split('-')
#         return (int(low)+int(high))/2
#     else:
#         return int(data)
# data['price'] = data['price'].apply(get_aver)

In [178]:
data

,area,title,house_size,bedroon_num,hall_num,bathroom_num,price,type,place
0,梁溪,整租·金星苑 1室1厅 南,65.0,1,1,1,1300.0,整租,金星苑
1,梁溪,独栋·柚米国际社区 柚米(无锡三阳广场店) 一居清新风 1室1厅,19.5,1,1,1,2072.5,独栋,柚米国际社区
2,梁溪,整租·华新苑 3室1厅 南,125.0,3,1,1,600.0,整租,华新苑
3,梁溪,独栋·柚米国际社区 酒吧街店 宜家风 1室1厅,27.0,1,1,1,2180.0,独栋,柚米国际社区
4,梁溪,整租·华新苑 1室2厅 南,48.0,1,2,1,1200.0,整租,华新苑
...,...,...,...,...,...,...,...,...,...
13084,江阴市,整租·大桥一村南区 1室1厅 南,45.0,1,1,1,1000.0,整租,大桥一村南区
13085,江阴市,整租·锦隆三村南区 1室1厅 南,20.0,1,1,1,800.0,整租,锦隆三村南区
13086,江阴市,整租·文定三村 3室1厅 南,93.0,3,1,1,650.0,整租,文定三村
13087,江阴市,合租·定波新村南区 1居室 南卧,30.0,1,1,0,850.0,合租,定波新村南区


In [179]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13089 entries, 0 to 13088
Data columns (total 9 columns):
area            13089 non-null object
title           13089 non-null object
house_size      13089 non-null float64
bedroon_num     13089 non-null int64
hall_num        13089 non-null int64
bathroom_num    13089 non-null int64
price           13089 non-null float64
type            13089 non-null object
place           13089 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 920.4+ KB


In [180]:
data.describe()

,house_size,bedroon_num,hall_num,bathroom_num,price
count,13089.000000,13089.000000,13089.000000,13089.000000,13089.000000
mean,84.444877,2.141034,1.331271,1.141187,2222.476354
std,71.853600,0.898401,0.592821,0.416379,994.885353
min,9.000000,0.000000,0.000000,0.000000,300.000000
25%,56.000000,1.000000,1.000000,1.000000,1700.000000
50%,88.000000,2.000000,1.000000,1.000000,2100.000000
75%,100.000000,3.000000,2.000000,1.000000,2600.000000
max,6000.000000,8.000000,8.000000,4.000000,25000.000000


In [203]:
#修改列名
data.rename(columns={'bedroon_num':'bedroom_num'},inplace=True) 

,level_0,index,area,title,house_size,bedroom_num,hall_num,bathroom_num,price,type,place
0,0,0,梁溪,整租·金星苑 1室1厅 南,65.0,1,1,1,1300.0,整租,金星苑
1,1,1,梁溪,独栋·柚米国际社区 柚米(无锡三阳广场店) 一居清新风 1室1厅,19.5,1,1,1,2072.5,独栋,柚米国际社区
2,2,2,梁溪,整租·华新苑 3室1厅 南,125.0,3,1,1,600.0,整租,华新苑
3,3,3,梁溪,独栋·柚米国际社区 酒吧街店 宜家风 1室1厅,27.0,1,1,1,2180.0,独栋,柚米国际社区
4,4,4,梁溪,整租·华新苑 1室2厅 南,48.0,1,2,1,1200.0,整租,华新苑
...,...,...,...,...,...,...,...,...,...,...,...
13060,13072,13084,江阴市,整租·大桥一村南区 1室1厅 南,45.0,1,1,1,1000.0,整租,大桥一村南区
13061,13073,13085,江阴市,整租·锦隆三村南区 1室1厅 南,20.0,1,1,1,800.0,整租,锦隆三村南区
13062,13074,13086,江阴市,整租·文定三村 3室1厅 南,93.0,3,1,1,650.0,整租,文定三村
13063,13075,13087,江阴市,合租·定波新村南区 1居室 南卧,30.0,1,1,0,850.0,合租,定波新村南区


In [205]:
#查看室为零的数据,发现室卫厅都是0,全删掉
data[data['bedroom_num']==0]
data.drop(index=(data[data['bedroom_num']==0].index),inplace=True)
data.reset_index(drop=True,inplace=True)

In [242]:
# 删除面积大于500的数据
data.drop(index=(data[(data['house_size']>500)&(data['house_size']<10)].index),inplace=True)

# 删除合租尺寸大于100的数据
data.drop(index=(data[(data['type']=='合租') & (data['house_size']>100)].index),inplace=True)
data.describe()

,level_0,index,house_size,bedroom_num,hall_num,bathroom_num,price
count,13048.000000,13048.000000,13048.000000,13048.000000,13048.000000,13048.000000,13048.000000
mean,6537.269237,6542.862814,83.017397,2.141708,1.332235,1.141324,2223.602315
std,3773.517397,3777.131300,36.517931,0.895086,0.588901,0.414111,992.704523
min,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,300.000000
25%,3270.750000,3273.750000,56.000000,1.000000,1.000000,1.000000,1700.000000
50%,6536.500000,6540.500000,88.000000,2.000000,1.000000,1.000000,2100.000000
75%,9808.250000,9817.250000,100.000000,3.000000,2.000000,1.000000,2600.000000
max,13076.000000,13088.000000,480.000000,8.000000,5.000000,4.000000,25000.000000


In [208]:
import pymysql
#连接数据库 127.0.0.1
conn = pymysql.connect(host='localhost',user='root',passwd='9483624ljk',db='Renting',port=3306,charset='utf8mb4')
#游标对象(操纵数据库必须使用游标提供了很多操作数据库的方法),比如执行sql
cursor = conn.cursor()
for i in range(0,data['area'].count()):
    #执行sql语句
    sql = "insert into Renting values('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(data['area'][i],data['title'][i],data['house_size'][i],data['bedroom_num'][i],data['hall_num'][i],data['bathroom_num'][i],data['price'][i],data['type'][i],data['place'][i])
    #执行sql语句
    cursor.execute(sql)
    #提交事务(查询可以不用使用)
    conn.commit()
#先关闭游标
cursor.close()
#关闭数据库
conn.close()

In [210]:
# 保存到excel
data.to_csv('Renting.xlsx',index=False)

In [211]:
from pyecharts import options as opts
from pyecharts.charts import *
from pyecharts.commons.utils import JsCode

In [237]:
#1室尺寸价格散点图
# 删除整租室大于1的数据
data1 = data.drop(index=(data[(data['type']=='整租')&(data['bedroom_num']>1)].index))
data1.reset_index(drop=True,inplace=True)
data1.describe()

,level_0,index,house_size,bedroom_num,hall_num,bathroom_num,price
count,3558.000000,3558.000000,3558.000000,3558.000000,3558.000000,3558.000000,3558.000000
mean,5452.793142,5457.506745,43.921585,1.022203,0.818156,0.955031,1637.399382
std,3969.174577,3972.874117,17.927286,0.240212,0.440866,0.250277,611.132769
min,0.000000,0.000000,10.000000,1.000000,0.000000,0.000000,300.000000
25%,1872.250000,1874.250000,32.125000,1.000000,1.000000,1.000000,1200.000000
50%,4525.500000,4528.500000,45.000000,1.000000,1.000000,1.000000,1650.000000
75%,9557.500000,9565.500000,53.000000,1.000000,1.000000,1.000000,2000.000000
max,13076.000000,13088.000000,400.000000,5.000000,3.000000,3.000000,10000.000000


In [233]:
scatter = (Scatter(init_opts=opts.InitOpts(theme='dark'))
           .add_xaxis(data1['house_size'])
           .add_yaxis('房租',data1['price'])
           .set_series_opts(label_opts=opts.LabelOpts(is_show=False),markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='max',name='最大值'),]))
           .set_global_opts(legend_opts=opts.LegendOpts(is_show=False),#不显示图例
                            title_opts=opts.TitleOpts(title='无锡租房 房租-尺寸 散点图'),
                            xaxis_opts=opts.AxisOpts(
                                name='尺寸',
                                # 设置坐标轴为数值类型
                                type_='value',
                                # 不显示分割线
                                splitline_opts=opts.SplitLineOpts(is_show=False)),
                            yaxis_opts=opts.AxisOpts(
                                name='总价',
                                name_location='middle',
                                type_='value',
                                # 默认为False表示起始为0
                                is_scale=True,
                                splitline_opts=opts.SplitLineOpts(is_show=False),),
                            # 通过颜色来度量房租
                            visualmap_opts=opts.VisualMapOpts(is_show=True,type_='color',min_=0,max_=3000)))
scatter.render_notebook()

In [214]:
data1[data1['price']==10000]

,level_0,index,area,title,house_size,bedroom_num,hall_num,bathroom_num,price,type,place
3324,11795,11807,新吴,整租·金源国际大厦 1室0厅 南,220.0,1,0,0,10000.0,整租,金源国际大厦


In [240]:
# 各行政区1室租房均价
# 整租大于一室的房租除以室的个数
data2 = data
data2[data2['house_size']<=10]
# if (data2['type']=='整租')&(data2['bedroom_num']>1):
#     data2['price'] = (data2['price'])/(data2['bedroom_num'])
# data2 = data.groupby(['area'])['price'].mean().reset_index()
# data2['area'] = temp['area'].map(lambda x:x+'区' if (x!='江阴市')&(x!='宜兴市') else x)


,level_0,index,area,title,house_size,bedroom_num,hall_num,bathroom_num,price,type,place
530,531,531,梁溪,整租·沁园新村 1室0厅 南,10.0,1,0,1,400.0,整租,沁园新村
3213,3215,3218,滨湖,整租·融创·瑷颐湾 1室1厅 北,10.0,1,1,2,820.0,整租,融创
6106,6110,6114,锡山,整租·东亭蓝郡 1室0厅 南,10.0,1,0,0,500.0,整租,东亭蓝郡
6175,6179,6183,锡山,整租·锡州小区 1室1厅 南,10.0,1,1,1,550.0,整租,锡州小区
6266,6271,6275,锡山,整租·隆亭新苑 1室0厅 南,10.0,1,0,0,1200.0,整租,隆亭新苑
7112,7117,7122,锡山,整租·隆亭新苑 1室1厅 南,10.0,1,1,1,900.0,整租,隆亭新苑
12280,12288,12300,新吴,整租·中邦城市花园 1室0厅 南,10.0,1,0,0,650.0,整租,中邦城市花园
12680,12688,12700,江阴市,整租·滨江别墅小区 3室2厅 南,9.0,3,2,1,500.0,整租,滨江别墅小区


In [219]:
data_pair = [(row['area'],round(row['price'],1)) for _,row in temp.iterrows()]

map_ = (Map(init_opts=opts.InitOpts(theme='dark'))
       .add('房租均价',data_pair,'无锡',is_roam=False)
       .set_series_opts(label_opts=opts.LabelOpts(is_show=Tree))
       .set_global_opts(
           title_opts=opts.TitleOpts(title='无锡各地区租房均价'),
           visualmap_opts=opts.VisualMapOpts(min_=300,max_=3000)
       ))
map_.render_notebook()

In [ ]:
# 均价最高10小区
temp = data1.groupby(['place'])['price'].agg(['mean','count']).reset_index()

data_pair = sorted([row['place'],round(row['mean'],1) )

In [ ]:
# 均价最高10地段
# 户型分布
# 词云图